# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=Layouts.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 Layouts

This notebook contains structure layouts to load the sometimes-opaque data blobs from Solana accounts.

The idea is to have one data-encapsulating class (in [BaseModel](BaseModel.ipynb)) for each type, as well as one or more LAYOUT structures. So for example `Group` loads `GROUP` but in future will also be able to load `GROUP_V2`.

The general approach is:
* Define one (or more) layouts to read in the data blob
* Use the data from the data blob to construct a more useful strongly-typed object

So for example `GROUP` is defined below but it's a low-level dependency. In general, code should depend and work with the `Group` class, not the `GROUP` structure.


In [ ]:
import construct
import datetime

from decimal import Decimal
from solana.publickey import PublicKey

from Constants import NUM_MARKETS, NUM_TOKENS


# Adapters

These are adapters for the construct package to simplify our struct declarations.

## DecimalAdapter class

A simple construct `Adapter` that lets us use `Decimal`s directly in our structs.

In [ ]:
class DecimalAdapter(construct.Adapter):
    def __init__(self, size: int = 8):
        construct.Adapter.__init__(self, construct.BytesInteger(size, swapped=True))

    def _decode(self, obj, context, path) -> Decimal:
        return Decimal(obj)

    def _encode(self, obj, context, path) -> int:
        # Can only encode int values.
        return int(obj)


## Float64Adapter class

Some numbers are packaged as 16-bytes to represent a `float`. The way to get the `float` is to take the 16-byte int value and divide it by 2 to the power 64. In Javascript this would be:
```
return intValue / Math.pow(2, 64);
```

This is a simple construct `Adapter` that lets us use these float values directly in our structs.

In [ ]:
class Float64Adapter(construct.Adapter):
    _divisor = Decimal(2 ** 64)

    def __init__(self, size: int = 16):
        construct.Adapter.__init__(self, construct.BytesInteger(size, swapped=True))

    def _decode(self, obj, context, path) -> Decimal:
        return Decimal(obj) / Float64Adapter._divisor

    def _encode(self, obj, context, path) -> bytes:
        return bytes(obj)


## PublicKeyAdapter

A simple construct `Adapter` that lets us use `PublicKey`s directly in our structs.

In [ ]:
class PublicKeyAdapter(construct.Adapter):
    def __init__(self):
        construct.Adapter.__init__(self, construct.Bytes(32))

    def _decode(self, obj, context, path) -> PublicKey:
        return PublicKey(obj)

    def _encode(self, obj, context, path) -> bytes:
        return bytes(obj)


## DatetimeAdapter

A simple construct `Adapter` that lets us load `datetime`s directly in our structs.

In [ ]:
class DatetimeAdapter(construct.Adapter):
    def __init__(self):
        construct.Adapter.__init__(self, construct.BytesInteger(8, swapped=True))

    def _decode(self, obj, context, path) -> datetime.datetime:
        return datetime.datetime.fromtimestamp(obj)

    def _encode(self, obj, context, path) -> bytes:
        return bytes(obj)

    

# Layout Structs

## SERUM_ACCOUNT_FLAGS

The SERUM_ prefix is because there's also `MANGO_ACCOUNT_FLAGS`.

Here's the [Serum Rust structure](https://github.com/project-serum/serum-dex/blob/master/dex/src/state.rs):
```
#[derive(Copy, Clone, BitFlags, Debug, Eq, PartialEq)]
#[repr(u64)]
pub enum AccountFlag {
    Initialized = 1u64 << 0,
    Market = 1u64 << 1,
    OpenOrders = 1u64 << 2,
    RequestQueue = 1u64 << 3,
    EventQueue = 1u64 << 4,
    Bids = 1u64 << 5,
    Asks = 1u64 << 6,
    Disabled = 1u64 << 7,
}
```

In [ ]:
SERUM_ACCOUNT_FLAGS = construct.BitsSwapped(
    construct.BitStruct(
        "initialized" / construct.Flag,
        "market" / construct.Flag,
        "open_orders" / construct.Flag,
        "request_queue" / construct.Flag,
        "event_queue" / construct.Flag,
        "bids" / construct.Flag,
        "asks" / construct.Flag,
        "disabled" / construct.Flag,
        construct.Padding(7 * 8)
    )
)


## MANGO_ACCOUNT_FLAGS

The MANGO_ prefix is because there's also `SERUM_ACCOUNT_FLAGS`.

The MANGO_ACCOUNT_FLAGS should be exactly 8 bytes.

Here's the [Mango Rust structure](https://github.com/blockworks-foundation/mango/blob/master/program/src/state.rs):
```
#[derive(Copy, Clone, BitFlags, Debug, Eq, PartialEq)]
#[repr(u64)]
pub enum AccountFlag {
    Initialized = 1u64 << 0,
    MangoGroup = 1u64 << 1,
    MarginAccount = 1u64 << 2,
    MangoSrmAccount = 1u64 << 3
}
```

In [ ]:
MANGO_ACCOUNT_FLAGS = construct.BitsSwapped(
    construct.BitStruct(
        "initialized" / construct.Flag,
        "group" / construct.Flag,
        "margin_account" / construct.Flag,
        "srm_account" / construct.Flag,
        construct.Padding(4 + (7 * 8))
    )
)


## INDEX

Here's the [Mango Rust structure](https://github.com/blockworks-foundation/mango/blob/master/program/src/state.rs):
```
#[derive(Copy, Clone)]
#[repr(C)]
pub struct MangoIndex {
    pub last_update: u64,
    pub borrow: U64F64,
    pub deposit: U64F64
}
```

In [ ]:
INDEX = construct.Struct(
    "last_update" / DatetimeAdapter(),
    "borrow" / Float64Adapter(),
    "deposit" / Float64Adapter()
);


## AGGREGATOR_CONFIG

Here's the [Flux Rust structure](https://github.com/blockworks-foundation/solana-flux-aggregator/blob/master/program/src/state.rs):
```
#[derive(Clone, Debug, BorshSerialize, BorshDeserialize, BorshSchema, Default, PartialEq)]
pub struct AggregatorConfig {
    /// description
    pub description: [u8; 32],

    /// decimals for this feed
    pub decimals: u8,

    /// oracle cannot start a new round until after `restart_relay` rounds
    pub restart_delay: u8,

    /// max number of submissions in a round
    pub max_submissions: u8,

    /// min number of submissions in a round to resolve an answer
    pub min_submissions: u8,

    /// amount of tokens oracles are reward per submission
    pub reward_amount: u64,

    /// SPL token account from which to withdraw rewards
    pub reward_token_account: PublicKey,
}
```

In [ ]:
AGGREGATOR_CONFIG = construct.Struct(
    "description" / construct.PaddedString(32, "utf8"),
    "decimals" / DecimalAdapter(1),
    "restart_delay" / DecimalAdapter(1),
    "max_submissions" / DecimalAdapter(1),
    "min_submissions" / DecimalAdapter(1),
    "reward_amount" / DecimalAdapter(),
    "reward_token_account" / PublicKeyAdapter()
);


## ROUND

Here's the [Flux Rust structure](https://github.com/blockworks-foundation/solana-flux-aggregator/blob/master/program/src/state.rs):
```
#[derive(Clone, Debug, BorshSerialize, BorshDeserialize, BorshSchema, Default, PartialEq)]
pub struct Round {
    pub id: u64,
    pub created_at: u64,
    pub updated_at: u64,
}
```

In [ ]:
ROUND = construct.Struct(
    "id" / DecimalAdapter(),
    "created_at" / DecimalAdapter(),
    "updated_at" / DecimalAdapter()
);


## ANSWER

Here's the [Flux Rust structure](https://github.com/blockworks-foundation/solana-flux-aggregator/blob/master/program/src/state.rs):
```
#[derive(Clone, Debug, BorshSerialize, BorshDeserialize, BorshSchema, Default, PartialEq)]
pub struct Answer {
    pub round_id: u64,
    pub median: u64,
    pub created_at: u64,
    pub updated_at: u64,
}
```

In [ ]:
ANSWER = construct.Struct(
    "round_id" / DecimalAdapter(),
    "median" / DecimalAdapter(),
    "created_at" / DatetimeAdapter(),
    "updated_at" / DatetimeAdapter()
);


## AGGREGATOR

Here's the [Flux Rust structure](https://github.com/blockworks-foundation/solana-flux-aggregator/blob/master/program/src/state.rs):
```
#[derive(Clone, Debug, BorshSerialize, BorshDeserialize, BorshSchema, Default, PartialEq)]
pub struct Aggregator {
    pub config: AggregatorConfig,
    /// is initialized
    pub is_initialized: bool,
    /// authority
    pub owner: PublicKey,
    /// current round accepting oracle submissions
    pub round: Round,
    pub round_submissions: PublicKey, // has_one: Submissions
    /// the latest answer resolved
    pub answer: Answer,
    pub answer_submissions: PublicKey, // has_one: Submissions
}
```

In [ ]:
AGGREGATOR = construct.Struct(
    "config" / AGGREGATOR_CONFIG,
    "initialized" / DecimalAdapter(1),
    "owner" / PublicKeyAdapter(),
    "round" / ROUND,
    "round_submissions" / PublicKeyAdapter(),
    "answer" / ANSWER,
    "answer_submissions" / PublicKeyAdapter()
);


## GROUP

Here's the [Mango Rust structure](https://github.com/blockworks-foundation/mango/blob/master/program/src/state.rs):
```
#[derive(Copy, Clone)]
#[repr(C)]
pub struct MangoGroup {
    pub account_flags: u64,
    pub tokens: [Pubkey; NUM_TOKENS],  // Last token is shared quote currency
    pub vaults: [Pubkey; NUM_TOKENS],  // where funds are stored
    pub indexes: [MangoIndex; NUM_TOKENS],  // to keep track of interest
    pub spot_markets: [Pubkey; NUM_MARKETS],  // pubkeys to MarketState of serum dex
    pub oracles: [Pubkey; NUM_MARKETS],  // oracles that give price of each base currency in quote currency
    pub signer_nonce: u64,
    pub signer_key: Pubkey,
    pub dex_program_id: Pubkey,  // serum dex program id

    // denominated in Mango index adjusted terms
    pub total_deposits: [U64F64; NUM_TOKENS],
    pub total_borrows: [U64F64; NUM_TOKENS],

    pub maint_coll_ratio: U64F64,  // 1.10
    pub init_coll_ratio: U64F64,  //  1.20

    pub srm_vault: Pubkey,  // holds users SRM for fee reduction

    /// This admin key is only for alpha release and the only power it has is to amend borrow limits
    /// If users borrow too much too quickly before liquidators are able to handle the volume,
    /// lender funds will be at risk. Hence these borrow limits will be raised slowly
    pub admin: Pubkey,
    pub borrow_limits: [u64; NUM_TOKENS],

    pub mint_decimals: [u8; NUM_TOKENS],
    pub oracle_decimals: [u8; NUM_MARKETS],
    pub padding: [u8; MANGO_GROUP_PADDING]
}
impl_loadable!(MangoGr
```

In [ ]:
GROUP_PADDING = 8 - (NUM_TOKENS + NUM_MARKETS) % 8

GROUP = construct.Struct(
    "account_flags" / MANGO_ACCOUNT_FLAGS,
    "tokens" / construct.Array(NUM_TOKENS, PublicKeyAdapter()),
    "vaults" / construct.Array(NUM_TOKENS, PublicKeyAdapter()),
    "indexes" / construct.Array(NUM_TOKENS, INDEX),
    "spot_markets" / construct.Array(NUM_MARKETS, PublicKeyAdapter()),
    "oracles" / construct.Array(NUM_MARKETS, PublicKeyAdapter()),
    "signer_nonce" / DecimalAdapter(),
    "signer_key" / PublicKeyAdapter(),
    "dex_program_id" / PublicKeyAdapter(),
    "total_deposits" / construct.Array(NUM_TOKENS, Float64Adapter()),
    "total_borrows" / construct.Array(NUM_TOKENS, Float64Adapter()),
    "maint_coll_ratio" / Float64Adapter(),
    "init_coll_ratio" / Float64Adapter(),
    "srm_vault" / PublicKeyAdapter(),
    "admin" / PublicKeyAdapter(),
    "borrow_limits" / construct.Array(NUM_TOKENS, DecimalAdapter()),
    "mint_decimals" / construct.Array(NUM_TOKENS, DecimalAdapter(1)),
    "oracle_decimals" / construct.Array(NUM_MARKETS, DecimalAdapter(1)),
    "padding" / construct.Array(GROUP_PADDING, construct.Padding(1))
);


## TOKEN_ACCOUNT

In [ ]:
TOKEN_ACCOUNT = construct.Struct(
    "mint" / PublicKeyAdapter(),
    "owner" / PublicKeyAdapter(),
    "amount" / DecimalAdapter(),
    "padding" / construct.Padding(93)
);


## OPEN_ORDERS

Trying to use the `OPEN_ORDERS_LAYOUT` and `OpenOrdersAccount` from `pyserum` just proved too probelmatic. (`OpenOrdersAccount` doesn't expose `referrer_rebate_accrued`, for instance.)

In [ ]:
OPEN_ORDERS = construct.Struct(
    construct.Padding(5),
    "account_flags" / SERUM_ACCOUNT_FLAGS,
    "market" / PublicKeyAdapter(),
    "owner" / PublicKeyAdapter(),
    "base_token_free" / DecimalAdapter(),
    "base_token_total" / DecimalAdapter(),
    "quote_token_free" / DecimalAdapter(),
    "quote_token_total" / DecimalAdapter(),
    "free_slot_bits" / DecimalAdapter(16),
    "is_bid_bits" / DecimalAdapter(16),
    "orders" / construct.Array(128, DecimalAdapter(16)),
    "client_ids" / construct.Array(128, DecimalAdapter()),
    "referrer_rebate_accrued" / DecimalAdapter(),
    "padding" / construct.Padding(7)
);


## MARGIN_ACCOUNT

Here's the [Mango Rust structure](https://github.com/blockworks-foundation/mango/blob/master/program/src/state.rs):
```
#[derive(Copy, Clone)]
#[repr(C)]
pub struct MarginAccount {
    pub account_flags: u64,
    pub mango_group: Pubkey,
    pub owner: Pubkey,  // solana pubkey of owner

    // assets and borrows are denominated in Mango adjusted terms
    pub deposits: [U64F64; NUM_TOKENS],  // assets being lent out and gaining interest, including collateral

    // this will be incremented every time an order is opened and decremented when order is closed
    pub borrows: [U64F64; NUM_TOKENS],  // multiply by current index to get actual value

    pub open_orders: [Pubkey; NUM_MARKETS],  // owned by Mango

    pub being_liquidated: bool,
    pub padding: [u8; 7] // padding to make compatible with previous MarginAccount size
    // TODO add has_borrows field for easy memcmp fetching
}
```

In [ ]:
MARGIN_ACCOUNT = construct.Struct(
    "account_flags" / MANGO_ACCOUNT_FLAGS,
    "mango_group" / PublicKeyAdapter(),
    "owner" / PublicKeyAdapter(),
    "deposits" / construct.Array(NUM_TOKENS, Float64Adapter()),
    "borrows" / construct.Array(NUM_TOKENS, Float64Adapter()),
    "open_orders" / construct.Array(NUM_MARKETS, PublicKeyAdapter()),
    "padding" / construct.Padding(8)
);


## FORCE_CANCEL

In [ ]:
FORCE_CANCEL = construct.Struct(
    "variant" / construct.Const(0xf, construct.BytesInteger(4, swapped=True)),
    "limit" / construct.BytesInteger(1, swapped=False)
);


## PARTIAL_LIQUIDATE

In [ ]:
PARTIAL_LIQUIDATE = construct.Struct(
    "variant" / construct.Const(0x10, construct.BytesInteger(4, swapped=True)),
    "maxDeposit" / DecimalAdapter()
);


# 🏃 Running

In [ ]:
if __name__ == "__main__":
    import base64
    import logging

    logging.getLogger().setLevel(logging.INFO)

    encoded = "AwAAAAAAAACCaOmpoURMK6XHelGTaFawcuQ/78/15LAemWI8jrt3SRKLy2R9i60eclDjuDS8+p/ZhvTUd9G7uQVOYCsR6+BhmqGCiO6EPYP2PQkf/VRTvw7JjXvIjPFJy06QR1Cq1WfTonHl0OjCkyEf60SD07+MFJu5pVWNFGGEO/8AiAYfduaKdnFTaZEHPcK5Eq72WWHeHg2yIbBF09kyeOhlCJwOoG8O5SgpPV8QOA64ZNV4aKroFfADg6kEy/wWCdp3fv0O4GJgAAAAAPH6Ud6jtjwAAQAAAAAAAADiDkkCi9UOAAEAAAAAAAAADuBiYAAAAACNS5bSy7soAAEAAAAAAAAACMvgO+2jCwABAAAAAAAAAA7gYmAAAAAAZFeDUBNVhwABAAAAAAAAABtRNytozC8AAQAAAAAAAABIBGiCcyaEZdNhrTyeqUY692vOzzPdHaxAxguht3JQGlkzjtd05dX9LENHkl2z1XvUbTNKZlweypNRetmH0lmQ9VYQAHqylxZVK65gEg85g27YuSyvOBZAjJyRmYU9KdCO1D+4ehdPu9dQB1yI1uh75wShdAaFn2o4qrMYwq3SQQEAAAAAAAAAAiH1PPJKAuh6oGiE35aGhUQhFi/bxgKOudpFv8HEHNCFDy1uAqR6+CTQmradxC1wyyjL+iSft+5XudJWwSdi7wvphsxb96x7Obj/AgAAAAAKlV4LL5ow6r9LMhIAAAAADvsOtqcVFmChDPzPnwAAAE33lx1h8hPFD04AAAAAAAA8YRV3Oa309B2wGwAAAAAA+yPBZRlZz7b605n+AQAAAACgmZmZmZkZAQAAAAAAAAAAMDMzMzMzMwEAAAAAAAAA25D1XcAtRzSuuyx3U+X7aE9vM1EJySU9KprgL0LMJ/vat9+SEEUZuga7O5tTUrcMDYWDg+LYaAWhSQiN2fYk7aCGAQAAAAAAgIQeAAAAAAAA8gUqAQAAAAYGBgICAAAA"
    decoded = base64.b64decode(encoded)

    group = GROUP.parse(decoded)
    print("\n\nThis is hard-coded, not live information!")
    print(group)
